In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
console_generations = {'genonecon':'Generation 1', 
                       '2gencon':'Generation 2',
                       '3gencon':'Generation 3', 
                       '4gencon':'Generation 4', 
                       '5gencon':'Generation 5', 
                       '6gencon':'Generation 6', 
                       '7gencon':'Generation 7', 
                       '8gencon':'Generation 8'}

url = 'https://en.wikipedia.org/wiki/List_of_first_generation_home_video_game_consoles'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table', {'id': console_generations.keys()})
console_tables = pd.read_html(str(tables))

url = 'https://en.wikipedia.org/wiki/List_of_home_video_game_consoles'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table', {'id': console_generations.keys()})
console_tables = console_tables + pd.read_html(str(tables))

new_col_names = {'Console':'console', 'Name':'console', 'Console / Series': 'console', 
 'Year':'release', 'Release date': 'release', 'Release date / Lifespan of the series':'release',
'Manufacturer':'manufacturer', 'Manufacturer / Country':'manufacturer'}

for n, generation in enumerate(console_tables):
    console_tables[n] = generation.rename(columns=new_col_names)
    console_tables[n] = console_tables[n][['console', 'release', 'manufacturer']]
    console_tables[n]['generation'] = list(console_generations.values())[n]

consoles = pd.concat(console_tables)
is_cancelled = consoles['release'].str.contains('cancelled', case=False)
consoles = consoles[is_cancelled == False]

consoles['release'] = consoles['release'].str.extract(r'(\d{4})')[0].astype('int64')

In [6]:
consoles.to_csv('data/consoles.csv', index=False)